In [1]:
%load_ext autoreload
%autoreload 2

Declaration of parameters (you must also add a tag for this cell - parameters)

In [2]:
#2. specify parameters
pipeline_params={
}
step_params={
}
substep_params={   
}

In [3]:
#3 define substep interface
from sinara.substep import NotebookSubstep, default_param_values, ENV_NAME, PIPELINE_NAME, ZONE_NAME, STEP_NAME, RUN_ID, ENTITY_NAME, ENTITY_PATH, SUBSTEP_NAME

substep = NotebookSubstep(pipeline_params, step_params, substep_params, **default_param_values("params/step_params.json"))

substep.interface(
    inputs =
    [ 
      { STEP_NAME: "model_train", ENTITY_NAME: "model"}
    ],
    
    tmp_outputs =
    [
        { ENTITY_NAME: "model" }
    ],
    
    outputs =
    [
        { ENTITY_NAME: "bento_service" }
    ],
)

substep.print_interface_info()

substep.exit_in_visualize_mode()

**INPUTS:**


[{'user.yolox_mmdet.test.model_train.model': '/data/home/jovyan/yolox_mmdet/test/model_train/run-23-10-26-230211/model'}]




**OUTPUTS:**


[{'user.yolox_mmdet.test.model_pack.bento_service': '/data/home/jovyan/yolox_mmdet/test/model_pack/run-23-10-27-073148/bento_service'}]




**TMP OUTPUTS:**


[{'tmp:user.yolox_mmdet.test.model_pack.model': '/data/tmp/user/yolox_mmdet/test/model_pack/run-23-10-27-073148/model'}]




![interface model_pack_interface.drawio](./imgs/model_pack_interface.drawio.png)

In [4]:
#4 get substep.interface
inputs = substep.inputs(step_name = "model_train")
tmp_outputs = substep.tmp_outputs()
outputs = substep.outputs()

print(f"{inputs.model=}")

print(f"{tmp_outputs.model=}")

print(f"{outputs.bento_service=}")

inputs.model='/data/home/jovyan/yolox_mmdet/test/model_train/run-23-10-26-230211/model'
tmp_outputs.model='/data/tmp/user/yolox_mmdet/test/model_pack/run-23-10-27-073148/model'
outputs.bento_service='/data/home/jovyan/yolox_mmdet/test/model_pack/run-23-10-27-073148/bento_service'


In [5]:
#5 run spark
from sinara.spark import SinaraSpark

spark = SinaraSpark.run_session(0)
SinaraSpark.ui_url()

Session is run


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/27 07:31:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


'http://localhost:4040'

### Loading a trained model from the model_train component 
(weights, configs)

In [6]:
from sinara.store import SinaraStore

# copy config from previos step to outputs
SinaraStore.copy_store_files_to_tmp(store_dir=inputs.model, tmp_dir=tmp_outputs.model)

In [7]:
!ls -lh {tmp_outputs.model}

total 279M
-rw-r----- 1 jovyan users 138M Oct 27 07:31 best_bbox_mAP_epoch_5.pth
-rw-r----- 1 jovyan users 1.2K Oct 27 07:31 config.json
-rw-r----- 1 jovyan users 7.1K Oct 27 07:31 last_cfg.py
-rw-r----- 1 jovyan users 138M Oct 27 07:31 latest.pth
-rw-r----- 1 jovyan users    0 Oct 27 07:31 _SUCCESS
-rw-r----- 1 jovyan users 109K Oct 27 07:31 test.jpg


### Reading the configuration file and adding parameters

In [8]:
import json
import os
import os.path as osp
config_fn = os.path.join(tmp_outputs.model, 'config.json')

with open(config_fn) as f_id:
    CONFIG = json.load(f_id)

In [9]:
CONFIG

{'FILTER_EMPTY_GT': False,
 'MIN_OBJECT_SIZE': 5,
 'train_coco_annotation': 'train_coco_annotations.json',
 'val_coco_annotation': 'val_coco_annotations.json',
 'test_coco_annotation': 'test_coco_annotations.json',
 'train_images': 'train_data',
 'val_images': 'eval_data',
 'test_images': 'test_data',
 'CLASSES': ['person', 'bicycle', 'car', 'motorcycle', 'bus', 'truck'],
 'train_config_parameters': {'MAX_SIZE': 640,
  'BATCH': 8,
  'WORKERS': 0,
  'SEED': 42,
  'EPOCH_COUNT': 5,
  'MODEL_NAME': 'yolox_s',
  'optimizer_lr': 0.001,
  'pretrain_weights': 'https://download.openmmlab.com/mmdetection/v2.0/yolox/yolox_s_8x8_300e_coco/yolox_s_8x8_300e_coco_20211121_095711-4592a793.pth',
  'Normalize': {'mean': [123.675, 116.28, 103.53],
   'std': [58.395, 57.12, 57.375],
   'to_rgb': True}},
 'config_file': 'last_cfg.py'}

In [10]:
DEVICE = "cuda:0"
MODEL_NAME = CONFIG["train_config_parameters"]["MODEL_NAME"]
CONFIG['work_dir'] = tmp_outputs.model
WORK_DIR = CONFIG['work_dir']
CONFIG['device'] = DEVICE

test_image_path = osp.join(tmp_outputs.model, "test.jpg")
assert osp.exists(test_image_path)

Selecting a weights file to convert to onnx format (best, latest epoch, etc.)

In [11]:
# Select model files
model_path = tmp_outputs.model
save_best  = False
files = [osp.join(model_path, file) for file in os.listdir(model_path)]

models_pth  = [file for file in files if '.pth' in file and not os.path.islink(file)]
best_models = [file for file in models_pth if 'best' in file]
latest_models = [file for file in models_pth if 'latest' in file]

assert len(best_models), 'model have not pth files.'

if save_best and len(best_models) > 0:
    models_pth = best_models
else:
    models_pth = latest_models
    
models_pth.sort(key=lambda file: osp.getmtime(file))

select_model = models_pth[-1]
cfg_model = osp.join(CONFIG['work_dir'], "last_cfg.py")

print(f"{select_model=}")

select_model='/data/tmp/user/yolox_mmdet/test/model_pack/run-23-10-27-073148/model/latest.pth'


In [12]:
CONFIG['config_model'] = cfg_model
CONFIG['torch_model'] = select_model
CONFIG['test_image_path'] = test_image_path

In [13]:
with open(config_fn, 'w') as f_id:
    json.dump(CONFIG, f_id, indent=4)

## Pack binary  model to bentoservice

In [14]:
from sinara.bentoml import save_bentoservice, load_bentoservice

In [26]:
# Clean model weiths - delete data of optimizer
import torch

state_dict = torch.load(CONFIG['torch_model'], map_location=torch.device('cpu'))

print(f"{state_dict.keys()=}")
if "optimizer" in state_dict:
    del state_dict["optimizer"]
    
clean_filename = osp.splitext(CONFIG['torch_model'])[0]+"_clean.pth"
torch.save(state_dict, clean_filename)

state_dict.keys()=dict_keys(['meta', 'state_dict', 'optimizer'])


In [30]:
from bento_service import Model_YOLOX_Pack
    
# initialize bento service
model_service = Model_YOLOX_Pack()
model_service.pack('model_name', MODEL_NAME)
serv_v = f"{outputs.fullname('bento_service')}.{substep.run_id}"
model_service.pack('service_version', serv_v)

# get model weights as binary file for bento_artifact
with open(clean_filename, "rb") as f_id:
    f_model = f_id.read()
    
# get model config as binary file for bento_artifact
with open(CONFIG['config_model'], "rb") as f_id:
    f_config = f_id.read()
    
model_service.pack('model', f_model)
model_service.pack('config', f_config)

# get test image as binary file for bento_artifact
with open(test_image_path, "rb") as f_id:
    f_test_image = f_id.read()
    
model_service.pack('test_image', f_test_image)

In [31]:
# save model as a bento pack
save_bentoservice(model_service, path=outputs.bento_service, service_version=serv_v)

[2023-10-27 07:47:29,139] WARNING - unknown package dependency for module: sinara
[2023-10-27 07:47:29,165] INFO - BentoService bundle 'Model_YOLOX_Pack:user.yolox_mmdet.test.model_pack.bento_service.run-23-10-27-073148' created at: /home/jovyan/work/yolox_mmdet/3_model_pack/tmp/run-23-10-27-073148/bento_service


In [32]:
#stop spark
SinaraSpark.stop_session()